In [46]:
# Dependencies 
import pandas as pd
import os
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [47]:
# Import alternate championhsip points tables
points_table_formula1 = pd.read_csv('points_systems_csv/2024_formula1_race_points_table.csv')
points_table_imsa = pd.read_csv('points_systems_csv/2024_imsa_race_points_table.csv')
points_table_imsa_qual = pd.read_csv('points_systems_csv/2024_imsa_qualifying_points_table.csv')


In [48]:
all_files_path = 'race_results_html/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

# Initialize a list to contain the race results 
race_result_data = []
i = 0

for race_result_file in race_results_files:
# Get City / Race Name
    i += 1
    j = 0
    city = race_result_file.split('.')[0]
    city = city[18:]
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')

# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        j += 1
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        if j == 1 and i == 1:
            row_data_with_city = row_data + ['City']
            race_result_data.append(row_data_with_city)
        elif j != 1:
            row_data_with_city = row_data + [city]
            race_result_data.append(row_data_with_city)
        else:
            pass

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_result_df['Rank'] = race_result_df['Rank'].astype('int')
    race_result_df['Starts'] = race_result_df['Starts'].astype('int')
    race_result_df['Laps'] = race_result_df['Laps'].astype('int')
    race_result_df['Laps Led'] = race_result_df['Laps'].astype('int')
    race_result_df['Points'] = race_result_df['Points'].astype('int')
    race_result_df['Pit stop'] = race_result_df['Pit stop'].astype('int64')
    race_result_df['Total time'] = pd.to_timedelta(race_result_df['Total time'])
    race_result_df['Total time in seconds'] = race_result_df['Total time'].dt.total_seconds()

    race_result_df = pd.merge(race_result_df, points_table_formula1, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa_qual, on='Starts', how='inner')
    race_result_df['Points (IMSA Scoring)'] = race_result_df['Points (IMSA Scoring)']+race_result_df['Points (IMSA Scoring) - Qualifying']
    race_result_df = race_result_df.drop(columns=['Points (IMSA Scoring) - Qualifying'])

    

race_result_df.head()

,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Total time in seconds,Points (F1 Scoring),Points (IMSA Scoring)
0,1,Scott McLaughlin,3,1,90,0 days 01:56:45.777300,90,Running,54,106.369,3,barber,7005.7773,25,385
1,2,Will Power,12,2,90,0 days 01:56:47.096700,90,Running,41,106.349,3,barber,7007.0967,18,352
2,3,Linus Lundqvist,8,19,90,0 days 01:56:48.219400,90,Running,36,106.332,3,barber,7008.2194,15,312
3,4,Felix Rosenqvist,60,5,90,0 days 01:56:50.288200,90,Running,33,106.301,2,barber,7010.2882,12,306
4,5,Alex Palou,10,10,90,0 days 01:56:51.146500,90,Running,31,106.288,2,barber,7011.1465,10,281


In [49]:
# Get a list of cities from race_result_df and convert it to a data frame
race_numbers = race_result_df.groupby('City')['City'].count()
race_numbers_df = pd.DataFrame(race_numbers)

# Define each race's race number
race_numbers_df.at['stpetersburg', 'Race Number'] = 1
race_numbers_df.at['longbeach', 'Race Number'] = 2
race_numbers_df.at['barber', 'Race Number'] = 3
race_numbers_df.at['indyrc', 'Race Number'] = 4
race_numbers_df.at['indy500', 'Race Number'] = 5
race_numbers_df.at['detroit', 'Race Number'] = 6
race_numbers_df.at['roadamerica', 'Race Number'] = 7
race_numbers_df.at['lagunaseca', 'Race Number'] = 8
race_numbers_df.at['midohio', 'Race Number'] = 9
race_numbers_df.at['iowa1', 'Race Number'] = 10
race_numbers_df.at['iowa2', 'Race Number'] = 11
race_numbers_df.at['toronto', 'Race Number'] = 12
race_numbers_df.at['gateway', 'Race Number'] = 13
race_numbers_df.at['portland', 'Race Number'] = 14
race_numbers_df.at['milwaukee1', 'Race Number'] = 15
race_numbers_df.at['milwaukee2', 'Race Number'] = 16
race_numbers_df.at['nashville', 'Race Number'] = 17

# Define track types for each race
race_numbers_df.at['stpetersburg', 'Track Type'] = 'Street Course'
race_numbers_df.at['longbeach', 'Track Type'] = 'Street Course'
race_numbers_df.at['barber', 'Track Type'] = 'Road Course'
race_numbers_df.at['indyrc', 'Track Type'] = 'Road Course'
race_numbers_df.at['indy500', 'Track Type'] = 'Oval'
race_numbers_df.at['detroit', 'Track Type'] = 'Street Course'
race_numbers_df.at['roadamerica', 'Track Type'] = 'Road Course'
race_numbers_df.at['lagunaseca', 'Track Type'] = 'Road Course'
race_numbers_df.at['midohio', 'Track Type'] = 'Road Course'
race_numbers_df.at['iowa1', 'Track Type'] = 'Oval'
race_numbers_df.at['iowa2', 'Track Type'] = 'Oval'
race_numbers_df.at['toronto', 'Track Type'] = 'Street Course'
race_numbers_df.at['gateway', 'Track Type'] = 'Oval'
race_numbers_df.at['portland', 'Track Type'] = 'Road Course'
race_numbers_df.at['milwaukee1', 'Track Type'] = 'Oval'
race_numbers_df.at['milwaukee2', 'Track Type'] = 'Oval'
race_numbers_df.at['nashville', 'Track Type'] = 'Oval'

#Clean the data frame
race_numbers_df['Race Number'] = race_numbers_df['Race Number'].astype('int64')
race_numbers_df = race_numbers_df.sort_values(by='Race Number')
race_numbers_df.rename(columns={'City': 'City2'}, inplace=True)
race_numbers_df.drop(columns=['City2'], inplace=True)
race_numbers_df = race_numbers_df.reset_index()

# Merge the data frames, thereby giving the race_result_df dataframe a race number 
race_result_df = pd.merge(race_result_df, race_numbers_df, on='City', how='inner')
race_result_df


,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Total time in seconds,Points (F1 Scoring),Points (IMSA Scoring),Race Number
0,1,Scott McLaughlin,3,1,90,0 days 01:56:45.777300,90,Running,54,106.369,3,barber,7005.7773,25,385,3
1,2,Will Power,12,2,90,0 days 01:56:47.096700,90,Running,41,106.349,3,barber,7007.0967,18,352,3
2,3,Linus Lundqvist,8,19,90,0 days 01:56:48.219400,90,Running,36,106.332,3,barber,7008.2194,15,312,3
3,4,Felix Rosenqvist,60,5,90,0 days 01:56:50.288200,90,Running,33,106.301,2,barber,7010.2882,12,306,3
4,5,Alex Palou,10,10,90,0 days 01:56:51.146500,90,Running,31,106.288,2,barber,7011.1465,10,281,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,23,Felix Rosenqvist,60,3,63,0 days 01:10:40.783500,63,Mechanical,7,95.517,2,toronto,4240.7835,0,110,12
462,24,Hunter McElrea,18,25,57,0 days 01:04:49.174800,57,Contact,6,94.233,2,toronto,3889.1748,0,76,12
463,25,Sting Ray Robb,41,27,13,0 days 00:17:46.888700,13,Mechanical,5,78.344,0,toronto,1066.8887,0,64,12
464,26,Agustin Canapino,78,10,4,0 days 00:06:13.867400,4,Contact,5,68.790,0,toronto,373.8674,0,71,12


In [50]:
print(race_result_df.dtypes)

race_result_df.drop(columns = 'Total time', inplace=True)

race_result_df.rename(columns = {
    "Race Number": "race_num",
    "City": "race_city",
    "Rank": "rank",
    "Driver": "driver",
    "Car No.": "car_no",
    "Starts": "start",
    "Laps": "laps",
    "Total time in seconds": "total_time",
    "Laps Led": "laps_led",
    "Status": "status",
    "Avg.Speed": "avg_speed",
    "Pit stop": "num_pit_stop",
    "Points": "points",
    "Points (F1 Scoring)": "points_f1",
    "Points (IMSA Scoring)": "points_IMSA"
}, inplace=True)

rev_order = ["race_num", "race_city", "rank", "driver", "car_no", "start", "laps", "total_time", "laps_led", "status", "avg_speed", "num_pit_stop", "points", "points_f1", "points_IMSA"]
reordered_race_result_df = race_result_df[rev_order]

reordered_race_result_df.to_csv('Resources/indydata_2024.csv', index=False, header=True)

Rank                               int32
Driver                            object
Car No.                           object
Starts                             int32
Laps                               int32
Total time               timedelta64[ns]
Laps Led                           int32
Status                            object
Points                             int32
Avg.Speed                         object
Pit stop                           int64
City                              object
Total time in seconds            float64
Points (F1 Scoring)                int64
Points (IMSA Scoring)              int64
Race Number                        int64
dtype: object
